In [ ]:
# Install PyTorch and TorchVision if not already installed
!pip install torch torchvision

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch.cuda.amp import autocast, GradScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image
import os

# Set device (assumes GPU like your H200 is available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your dataset
df = pd.read_csv('../scrape/psa_sales_190786_20250222_143954.csv')  # Replace with your CSV file path
image_dir = '../scrape/pictures'  # Replace with your image directory path
df['filename'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))

# Split into training and validation sets
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

# Encode the grade labels (e.g., 'PSA1' to 'PSA10') into integers
le = LabelEncoder()
le.fit(df['grade'])
train_df['label'] = le.transform(train_df['grade'])
val_df['label'] = le.transform(val_df['grade'])

# Optional: Compute class weights to handle imbalance
classes = np.unique(train_df['grade'])
class_weights = compute_class_weight('balanced', classes=classes, y=train_df['grade'])
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define transforms for training and validation
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define custom dataset class
class CardDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['filename']
        label = self.df.iloc[idx]['label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Create datasets and dataloaders
batch_size = 64  # Adjust based on your H200 GPU memory
train_dataset = CardDataset(train_df, transform=train_transform)
val_dataset = CardDataset(val_df, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Build the model with ResNet101
from torchvision.models import resnet101, ResNet101_Weights
model = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 10)
)
model = model.to(device)

# Freeze all layers except the final classifier (fc)
for name, param in model.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)  # Remove weight if not needed
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, min_lr=1e-6)

# Define validation function
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_acc = correct / total
    return val_loss, val_acc

# Training loop with early stopping (initial phase)
num_epochs = 50
patience = 5
best_val_loss = float('inf')
patience_counter = 0
scaler = GradScaler()  # For mixed precision training

history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    val_loss, val_acc = validate(model, val_loader, criterion)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('Early stopping')
            break

# Load best model and unfreeze layer4 and fc for fine-tuning
model.load_state_dict(torch.load('best_model.pth'))
for name, param in model.named_parameters():
    if 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# Reset for fine-tuning
best_val_loss = float('inf')
patience_counter = 0
history_fine = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    val_loss, val_acc = validate(model, val_loader, criterion)
    history_fine['train_loss'].append(train_loss)
    history_fine['val_loss'].append(val_loss)
    history_fine['val_accuracy'].append(val_acc)
    
    print(f'Fine-tune Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model_finetune.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('Early stopping')
            break

# Load the best fine-tuned model
model.load_state_dict(torch.load('best_model_finetune.pth'))

# Visualize training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['val_accuracy'] + history_fine['val_accuracy'], label='Val Accuracy')
plt.plot(history['train_loss'] + history_fine['train_loss'], label='Train Loss')  # Proxy since train acc not computed
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['val_loss'] + history_fine['val_loss'], label='Val Loss')
plt.plot(history['train_loss'] + history_fine['train_loss'], label='Train Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Save the trained model
torch.save(model.state_dict(), 'card_grader_model.pth')

# Define prediction function
def predict_grade(img_path, model, le, transform):
    model.eval()
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    predicted_class = predicted.item()
    predicted_grade = le.inverse_transform([predicted_class])[0]
    return predicted_grade

# Example prediction
new_card_path = '../scrape/pictures/cert_01443963.jpg'  # Replace with your test image path
predicted_grade = predict_grade(new_card_path, model, le, val_transform) # (expected 8)
print(f"Predicted PSA grade: {predicted_grade}")